Begin to implement chains and implement a pipeline 

In [ ]:
with open("job_desc.md") as f:
    description = f.read()
with open("resume.md") as f:
    resume = f.read()
title = 'django developer'

In [ ]:
from util import chunk_markdown, cut_sections

In [ ]:
sections = cut_sections(resume)

history = sections['history']['result']
chunks = chunk_markdown(history)

We need a function to only select job history from chunks. Any other headings such as summary, or education should be set aside.

We need to split the job section into 2 parts, the job experience description and the job metadata.

## Begin to process job history chunks

In [ ]:
tmp_chunks = chunks.copy()

for chunk in tmp_chunks:
    chunk.page_content = 'test'
print(tmp_chunks)

Implementing `beef_section_chain()`

In [ ]:
from beef import highlight_chain
from util import relevant_skills_chain

In [ ]:
from util import job_requirement_chain


requirements = job_requirement_chain().predict(desc=description)


In [ ]:
from langchain.chains import SequentialChain

beef_chain = SequentialChain(
    chains=[relevant_skills_chain(), highlight_chain()],
    input_variables=["title", "desc", "section", "requirements"],
    output_variables=["highlighted"],
    verbose=True
)

for chunk in chunks:
    print(beef_chain({"section": chunk.page_content, "title": title, "desc": description, "requirements": requirements}))

It seems that a router should be used to evaluate a job experience section. If a job experience section is not strong for a given job description, THAN the chains described in this notebook should be used. This chain should be called `BeefSectionChain`. It's intention should be to highlight the key skills needed for a given job description.

When processing the job experience section as a whole, a comparison should be made between the processed section, and the original section. The section that returns `"more appealing to a recruiter"` should be used as `Document.page_content`.

How can we build an excellent resume summary from the job experience sections?

First, each job experience will be filtered with a prompt such as `"what is one thing that a recruiter for x job needs to know"`. Then that list of facts will be reduced to 3. Then the original will be updated with `"make this appealing for a recruiter for x job"`. The two proposed sections will be compared.

Or maybe model can just condense, then choose 3-5 of the best lines.